In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
import  analysis.lib.purification.purify_pq as ppq

%matplotlib inline

thetas = ['pi/4','pi/5','pi/6','pi/8']

outcome_bins = np.arange(50,540,50)

In [ ]:
def calculate_ebit_rate(outcome_bins,apply_ROC = True):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
    
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    total_counts_p = psi_p_XX[0]+psi_p_YY[0]+psi_p_ZZ[0]
    total_counts_m = psi_m_XX[0]+psi_m_YY[0]+psi_m_ZZ[0]
    
    total_time = Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    rate_p = np.array(total_counts_p)/total_time
    rate_m = np.array(total_counts_m)/total_time
    
    ebit_rate_p = np.sum(rate_p*ebits_p)
    ebit_rate_p_u = np.sqrt(np.sum(rate_p**2*ebits_p_u**2))
    ebit_rate_m = np.sum(rate_m*ebits_m)
    ebit_rate_m_u = np.sqrt(np.sum(rate_m**2*ebits_m_u**2))
    
    avg_fidelity_p = np.sum(psi_p_F*total_counts_p)/np.sum(total_counts_p)
    avg_fidelity_p_u = np.sqrt(np.sum(psi_p_F_u**2*total_counts_p**2))/np.sum(total_counts_p)
    avg_fidelity_m = np.sum(psi_m_F*total_counts_p)/np.sum(total_counts_m)
    avg_fidelity_m_u = np.sqrt(np.sum(psi_m_F_u**2*total_counts_p**2))/np.sum(total_counts_m)
    
    rate_p = np.sum(rate_p)
    rate_m = np.sum(rate_m)
    
    return [rate_p,rate_m,ebit_rate_p,ebit_rate_p_u,ebit_rate_m,ebit_rate_m_u,avg_fidelity_p,avg_fidelity_p_u,avg_fidelity_m,avg_fidelity_m_u]

In [ ]:
vals = []

for theta in thetas:
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    Purify_XX.get_tstamps_and_offsets(contains = 'XX')
    Purify_YY.get_tstamps_and_offsets(contains = 'YY')
    Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')

#     ### load the data
    Purify_XX.load_raw_data()
    Purify_XX.correct_pq_times()
    Purify_YY.load_raw_data()
    Purify_YY.correct_pq_times()
    Purify_ZZ.load_raw_data()
    Purify_ZZ.correct_pq_times()
    
    vals.append(calculate_ebit_rate(outcome_bins = outcome_bins))
   
vals = np.transpose(vals)

## Calculations for other bounds

In [ ]:
TPQI_vis = 0.5765
TPQI_vis_u = 0.04

estimatedFirstClickProb = 0.000757 # At 50% superpos.
tailLT3 = 12.
tailLT4 = 6.
firstClickProbLT3 = (tailLT4/(tailLT4+tailLT3))*estimatedFirstClickProb
firstClickProbLT4 = (tailLT3/(tailLT4+tailLT3))*estimatedFirstClickProb

pBK = 0.5*firstClickProbLT3*firstClickProbLT4
estimatedRateBK = pBK / (7e-6)
estimatedEbitsBK= np.log2(1+TPQI_vis)

# print pBK
# print estimatedRateBK
# print estimatedEbitsBK

estimatedEbitRateBK = estimatedRateBK*estimatedEbitsBK


In [ ]:
thetas = [np.pi/4,np.pi/5,np.pi/6,np.pi/8]

rate_p = vals[0]
rate_m = vals[1]
ebit_rate_p = vals[2]
ebit_rate_p_u = vals[3]
ebit_rate_m = vals[4]
ebit_rate_m_u = vals[5]
avg_fidelity_p = vals[6]
avg_fidelity_p_u = vals[7]
avg_fidelity_m = vals[8]
avg_fidelity_m_u = vals[9]

super_posn_frac = np.sin(thetas)**2

total_ebit_rate = ebit_rate_p + ebit_rate_m
total_ebit_rate_u = np.sqrt(ebit_rate_p_u**2 + ebit_rate_m_u**2)

fig  = plt.figure()
ax = plt.subplot()
ax.set_xlabel('Superposn ms=0 frac.')
ax.set_ylabel('Ebit rate')
plt.errorbar(super_posn_frac,total_ebit_rate,total_ebit_rate_u,label = 'Pur')
plt.plot([0, 0.6], [estimatedEbitRateBK, estimatedEbitRateBK], label = 'BK')
plt.plot([0, 0.6], [estimatedRateBK, estimatedRateBK], label = 'BK perfect')
plt.xlim([0.1,0.55])
plt.ylim([0,plt.ylim()[1]])
plt.legend()
plt.show()
plt.close('all')

fig  = plt.figure()
ax = plt.subplot()
ax.set_xlabel('Superposn ms=0 frac.')
ax.set_ylabel('Raw rate (Hz)')
plt.plot(super_posn_frac,rate_m,label = '-')
plt.plot(super_posn_frac,rate_p,label = '+')
plt.xlim([0.1,0.55])
plt.legend()
plt.show()
plt.close('all')

fig  = plt.figure()
ax = plt.subplot()
ax.set_xlabel('Superposn ms=0 frac.')
ax.set_ylabel('Avg fidelity')
plt.errorbar(super_posn_frac,avg_fidelity_m,avg_fidelity_m_u,label = '-')
plt.errorbar(super_posn_frac,avg_fidelity_p,avg_fidelity_p_u,label = '+')
plt.xlim([0.1,0.55])
plt.legend()
plt.show()
plt.close('all')